In [1]:
import pandas as pd
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
from tqdm import tqdm
tqdm.pandas()
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
import matplotlib.pyplot as plt
import sys
import warnings
import folium
import pandas as pd
from folium import plugins
from folium import IFrame
if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [2]:
df1=pd.read_pickle('processed_DI_data.pkl')
df_lyft= df1[df1['word_check']=='lyft']
df_lyft['count']=[1 for i in range(len(df_lyft))]
df_lyft['cum_count']=df_lyft['count'].cumsum()
df_lyft['senti_score']=df_lyft.progress_apply(lambda x: analyzer.polarity_scores(x['cln_txt'])['compound'],axis=1)
df_lyft['cum_senti_score']=df_lyft['senti_score'].cumsum()

df_uber= df1[df1['word_check']=='uber']
df_uber['count']=[1 for i in range(len(df_uber))]
df_uber['cum_count']=df_uber['count'].cumsum()
df_uber['senti_score']=df_uber.progress_apply(lambda x: analyzer.polarity_scores(x['cln_txt'])['compound'],axis=1)
df_uber['cum_senti_score']=df_uber['senti_score'].cumsum()

df_taxi= df1[df1['word_check']=='taxi']
df_taxi['count']=[1 for i in range(len(df_taxi))]
df_taxi['cum_count']=df_taxi['count'].cumsum()
df_taxi['senti_score']=df_taxi.progress_apply(lambda x: analyzer.polarity_scores(x['cln_txt'])['compound'],axis=1)
df_taxi['cum_senti_score']=df_taxi['senti_score'].cumsum()

100%|██████████████████████████████████████████████████████████████████████████████| 345/345 [00:00<00:00, 3511.03it/s]


# Spatial Visualization

In [12]:
m = folium.Map([27.5, -81.5], zoom_start=4)
feature_lyft_point = folium.FeatureGroup(name='Sample Lyft Tweet')
feature_uber_point = folium.FeatureGroup(name='Sample Uber Tweet')
feature_lyft_hm = folium.FeatureGroup(name='Lyft Location Heatmap')
feature_uber_hm = folium.FeatureGroup(name='Uber Location Heatmap')

data=[[row['lat'],row['lon']] for index, row in df_lyft.iterrows()]
hm = plugins.HeatMap(data).add_to(feature_lyft_hm)


data=[[row['lat'],row['lon']] for index, row in df_uber.iterrows()]
hm = plugins.HeatMap(data).add_to(feature_uber_hm)

In [13]:
lat_lon=[(float(row['lat']),float(row['lon'])) for index, row in df_lyft.iterrows()]
for i in lat_lon[:10]:
    df = df_lyft[(df_lyft['lat']==i[0]) & (df_lyft['lon']==i[1])]
    df=df.reset_index()
#    df['text']=df.apply(lambda x: text_cleaner(x.text), axis=1)
    df=df[['time','text']].astype(str)
    df=df.set_index('time')
    html = df.to_html()
    popup = folium.Popup(html,max_width=1500)    
    folium.Marker(i, popup=popup).add_to(feature_lyft_point)

In [14]:
lat_lon=[(float(row['lat']),float(row['lon'])) for index, row in df_uber.iterrows()]
for i in lat_lon[:10]:
    df = df_uber[(df_uber['lat']==i[0]) & (df_uber['lon']==i[1])]
    df=df.reset_index()
#    df['text']=df.apply(lambda x: text_cleaner(x.text), axis=1)
    df=df[['time','text']].astype(str)
    df=df.set_index('time')
    html = df.to_html()
    popup = folium.Popup(html,max_width=1500)    
    folium.Marker(i, popup=popup,color='red').add_to(feature_uber_point)

In [15]:
m.add_child(feature_lyft_point)
m.add_child(feature_uber_point)
m.add_child(feature_lyft_hm)
m.add_child(feature_uber_hm)
m.add_child(folium.map.LayerControl())
m

In [8]:
m.save('Spatial_Vis_DI.html')